In [ ]:
# imports
!pip install mirdata
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import mirdata
import librosa
import librosa.display
import matplotlib.pyplot as plt
from torch import nn

In [3]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
# Convert data to torch tensors
# in other words, write your data loader
class Data(Dataset):
    def __init__(self, dataset_name, split_strings, hop_length=256, n_fft=512, dur=4.9):
        self.dataset = mirdata.initialize(dataset_name, data_home='/content/drive/MyDrive/egfxset')
        # self.dataset.download(partial_download=['hallReverb'])
        self.split_strings = split_strings # the guitar strings that will be used by the dataloader
        self.track_ids = []
        # filter out the strings that should not be
        # in the dataset split
        # also filter out any effects not featured in
        # labels dict
        for t in self.dataset.track_ids:
          string_number = int(t.split('/')[-1][0])
          if string_number in self.split_strings:
            self.track_ids.append(t)
        self.dur=dur
        self.hop_length=hop_length
        self.n_fft = n_fft

    def __getitem__(self, index):
        # write the getitem method

        # load the audio signal

        # compute the absolute-valued spectrogram

        # return the spectrogram, flattened and normalized by its largest value
        return

    def __len__(self):
        return len(self.track_ids)

DATASET_NAME = 'egfxset'
TRAIN_STRINGS = [0,1,2,3,4]
TEST_STRINGS = [5]
HOP_LENGTH = 512
N_FFT = 1024

batch_size = 4

# Instantiate training and test data
train_data = Data(DATASET_NAME, TRAIN_STRINGS, n_fft=N_FFT, hop_length=HOP_LENGTH)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

test_data = Data(DATASET_NAME, TEST_STRINGS, n_fft=N_FFT, hop_length=HOP_LENGTH)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

# Check it's working
for batch, X in enumerate(test_loader):
    print(f"Batch: {batch+1}")
    print(f"X shape: {X.shape}")
    break


# """
# Batch: 1
# X shape: torch.Size([64, 2])
# y shape: torch.Size([64])
# """

In [5]:
# test the griffinlim algorithm with examples from your dataset
# WARNING: it assumes that the spectrogram is NOT flattened (you'll need to 'unflatten')
from IPython.display import Audio, display
y = train_data[0].numpy()
y_inv = librosa.griffinlim(y, n_fft=N_FFT, hop_length=HOP_LENGTH)

display(Audio(y_inv, rate=48000))

In [6]:
# define the variational autoencoder
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim, input_length):
        super(VAE, self).__init__()

        self.input_shape = input_length

        # encoder part
        self.fc1 = nn.Linear(# your code here
        self.fc2 = nn.Linear(# your code here
        self.fc31 = nn.Linear(# your code here (mu)
        self.fc32 = nn.Linear(# your code here (sigma)
        # decoder part
        self.fc4 = nn.Linear(# your code here
        self.fc5 = nn.Linear(# your code here
        self.fc6 = nn.Linear(# your code here

    def encoder(self, x):
        h = F.relu(# your code here
        h = F.relu(# your code here
        return self.fc31(h), self.fc32(h) # mu, log_var

    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample

    def decoder(self, z):
        h = # your code here
        h = # your code here
        return F.relu(self.fc6(h))

    def forward(self, x):
        mu, log_var = self.encoder(# your code here
        z = self.sampling(# your code here
        return self.decoder(z), mu, log_var

# build model
INPUT_DIMENSIONALITY = # your code here
vae = VAE(x_dim=INPUT_DIMENSIONALITY, h_dim1= 512, h_dim2=256, z_dim=128)
if torch.cuda.is_available():
    vae.cuda()

In [8]:
# no need to do anything here, just run it
optimizer = torch.optim.Adam(vae.parameters(), lr=0.001)
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.mse_loss(recon_x, x.view(-1, INPUT_DIMENSIONALITY), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [9]:
# no need to do anything here, just run it
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, max) in enumerate(test_loader):
        data = data.cuda()
        optimizer.zero_grad()

        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)

        loss.backward()
        train_loss += loss.item()
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [10]:
# no need to do anything here, just run it
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for (data,i) in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)

            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
# no need to do anything here, just run it
# are you ready?
for epoch in range(1, 51):
    train(epoch)
    test()

In [44]:
# no need to do anything here, just run it
# this will sample your latent space and generate audio
with torch.no_grad():
    z = torch.randn(1, 128).cuda()
    sample = vae.decoder(z).cuda()

sample = sample.view([1, N_FFT//2 + 1,-1])
y_inv = librosa.griffinlim(sample.cpu().numpy(), n_fft=N_FFT, hop_length=HOP_LENGTH)
display(Audio(y_inv, rate=48000))